In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Setup meshcat

In [4]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
from meshcat import Visualizer
from pydrake.all import ConnectMeshcatVisualizer

proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

do_viz = True

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


# Build plant

In [5]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
                                      delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-6,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1
                        /2)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


geomA=iiwa::link7, geomB=shelves::top
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::shelf_upper
0
geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553085918943
snopt_example=[0.55420806 0.09553807 0.18634565], growth = 0.1169996008726882
snopt_example=[ 0.30833471 -0.27841844  0.24117495], growth 

snopt_example=[ 0.17287864 -1.02198055 -0.1322734 ], growth = 2.336728594175156
snopt_example=[ 0.17287844 -1.02198386 -0.13227868], growth = 2.3366924367858246
snopt_example=[ 0.17287825 -1.02198717 -0.13228396], growth = 2.336656279779419
snopt_example=[ 0.17287806 -1.02199048 -0.13228923], growth = 2.3366201231559813
snopt_example=[ 0.17287786 -1.02199379 -0.13229451], growth = 2.336583966915512
snopt_example=[ 0.17287767 -1.0219971  -0.13229978], growth = 2.336547811057983
snopt_example=[ 0.17287748 -1.02200041 -0.13230506], growth = 2.336511655583408
snopt_example=[ 0.17287728 -1.02200372 -0.13231034], growth = 2.3364755004918294
snopt_example=[ 0.17287709 -1.02200703 -0.13231561], growth = 2.3364393457831625
snopt_example=[ 0.1728769  -1.02201034 -0.13232089], growth = 2.3364031914574923
snopt_example=[ 0.17287671 -1.02201366 -0.13232616], growth = 2.3363670375147336
snopt_example=[ 0.17287651 -1.02201697 -0.13233144], growth = 2.336330883954986
snopt_example=[ 0.17287632 -1.0220

snopt_example=[ 0.17285678 -1.02235419 -0.13286924], growth = 2.332645237847771
snopt_example=[ 0.17285658 -1.0223575  -0.13287452], growth = 2.3326091237822197
snopt_example=[ 0.17285639 -1.02236081 -0.1328798 ], growth = 2.332573010099509
snopt_example=[ 0.1728562  -1.02236413 -0.13288508], growth = 2.3325368967996525
snopt_example=[ 0.172856   -1.02236744 -0.13289036], growth = 2.3325007838826366
snopt_example=[ 0.17285577 -1.02236976 -0.13289477], growth = 2.332464671356078
snopt_example=[ 0.17285558 -1.02237308 -0.13290006], growth = 2.3324285592871092
snopt_example=[ 0.17285538 -1.0223764  -0.13290534], growth = 2.332392447600782
snopt_example=[ 0.17285519 -1.02237972 -0.13291062], growth = 2.332356336297039
snopt_example=[ 0.172855   -1.02238303 -0.13291591], growth = 2.3323202253758097
snopt_example=[ 0.1728548  -1.02238635 -0.13292119], growth = 2.3322841148371793
snopt_example=[ 0.17285461 -1.02238967 -0.13292647], growth = 2.332248004681176
snopt_example=[ 0.17285442 -1.0223

In [9]:
# plot regions
if do_viz:
    viz_utils.plot_regions(vis2, regions, ellipses)

    # plot seedpoints
    for i in range(seed_points.shape[0]):
        vis2['iris']['seedpoints']["seedpoint"+str(i)].set_object(
                    meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x0FB900))
        vis2['iris']['seedpoints']["seedpoint"+str(i)].set_transform(
                    meshcat.transformations.translation_matrix(seed_points[i,:]))
    

## Marching cubes in t space

In [10]:
#evaluate constraints
def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q

def convert_q_to_t(q):
    return np.tan(np.divide(q,2))

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)
def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def eval_cons_rational(t0, t1, t2, c, tol):
    q = convert_t_to_q(np.array([t0, t1, t2]).reshape(1,-1)).squeeze() 
    return col_func_handle(*q)
   
col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)


In [11]:
#marching cubes
if do_viz:
    N = 50
    vertices, triangles = mcubes.marching_cubes_func(tuple(iris_generator.t_lower_limits), 
                                                     tuple(iris_generator.t_upper_limits),
                                                     N, N, N, col_func_handle_rational, 0.5)

    vis2["collision_constraint"].set_object(
                meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
                meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))

In [17]:
certification_problems = iris_generator.initalize_certifier( plane_order = 1, strict_pos_tol = 1e-6)



time to create maps: : 0.7735
Time to initialize region program: 0.7737
time to create Region Certification Problem: : 0.377

time to create maps: : 0.7569
Time to initialize region program: 0.757
time to create Region Certification Problem: : 0.3779

time to create maps: : 0.9336
Time to initialize region program: 0.9336
time to create Region Certification Problem: : 1.3881

time to create maps: : 1.0621
Time to initialize region program: 1.0621
time to create Region Certification Problem: : 2.252

time to create maps: : 1.13
Time to initialize region program: 1.13
time to create Region Certification Problem: : 2.7814

time to create maps: : 0.874
Time to initialize region program: 0.8741
time to create Region Certification Problem: : 0.8865



In [19]:
for i, (region, problem) in enumerate(certification_problems.items()):
    t0 = time.time()
    eps = 1e-1*np.ones(region.b().shape[0])
    success, collision_pairs_to_plane_result_map, geom_id_and_vert_to_pos_poly_and_multiplier_result_map,\
    geom_id_and_vert_to_pos_poly_variable_and_multiplier_result_map = \
    problem.attempt_certification_for_epsilon(eps)
    print(f"Region {i+1}/{len(certification_problems.keys())} certfied = {success}")
    t1 = time.time()
    print(f"Completed in {t1-t0}s")
    print()

Region 1/6 certfied = False
Completed in 0.8218302726745605s

Region 2/6 certfied = False
Completed in 0.7924916744232178s

Region 3/6 certfied = True
Completed in 1.5574939250946045s

Region 4/6 certfied = True
Completed in 2.0424110889434814s

Region 5/6 certfied = True
Completed in 3.1264536380767822s

Region 6/6 certfied = True
Completed in 1.2487292289733887s



Notes:
Currently have build certifiaction problems from maps done. Now can solve. Current design strategy means we should not solve the programs in parallel since all the programs point
to the same variables. This saves a lot of memory and (hopefully) speeds up the construction substantially

In [14]:
problem = certification_problems[regions[1]]
self = problem

In [15]:
success, collision_pairs_to_plane_result_map, geom_id_and_vert_to_pos_poly_and_multiplier_result_map,\
geom_id_and_vert_to_pos_poly_variable_and_multiplier_result_map = \
problem.attempt_certification_for_epsilon(1e-2*np.ones(problem.region.b().shape[0]))
print(success)

False


In [16]:
problem.build_fixed_multiplier_problem(geom_id_and_vert_to_pos_poly_variable_and_multiplier_result_map)

problem.attempt_reduce_eps_in_certificate(geom_id_and_vert_to_pos_poly_variable_and_multiplier_result_map)

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
self.solver.Solve(self.certify_for_fixed_multiplier_prog).is_success()

In [ ]:
print(problem.certify_for_fixed_multiplier_prog)